# Exploring Quantum Newton Fractals

Quantum Newton Fractals offer a captivating window into the complex realm of quantum mechanics. These mesmerizing visualizations, generated through the powerful Newton-Raphson method, unveil the intricate patterns and dynamics of wave functions and probabilities.

In this journey of discovery, we embark on a voyage to unravel the mysteries of Quantum Newton Fractals using the versatile tools of Python. With Matplotlib's graphical prowess and SymPy's symbolic might, we delve deep into the heart of these fractals, uncovering their beauty and complexity.

Join us as we manipulate parameters in real-time, unleashing the power of interactive widgets to shape and explore the wondrous Quantum Newton Fractals. Let's embark on this enchanting exploration together, where mathematics meets art in the realm of quantum mechanics.


The provided Python code defines functions to generate and visualize a Quantum Newton Fractal using Matplotlib and SymPy. Here's a breakdown of the code:

1. **Imports**:
   - `matplotlib.pyplot` as `plt`
   - `numpy` as `np`
   - `sympy` as `sp`
   - `interactive`, `FloatSlider`, `Text`, `IntSlider`, `VBox`, `HBox` from `ipywidgets`

2. **Function Definitions**:
   - `compute_fractal`: Computes the fractal using the Newton-Raphson method.
   - `create_meshgrid`: Creates a meshgrid for given x and y limits, width, and height.
   - `newton_fractal`: Generates the Newton Fractal using the provided functions.
   - `generate_quantum_newton_fractal`: Generates the Quantum Newton Fractal and displays it.

3. **Interactive Interface Elements**:
   - Text input for custom function definition.
   - Sliders for adjusting limits, width, height, max iterations, and tolerance.
   
4. **Interactive Plot**:
   - Displays the Quantum Newton Fractal based on the provided parameters.

## Dependencies

In [36]:
%%capture
!pip install sympy
!pip install ipywidgets
!pip numba

# Imports

In [37]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from mpl_toolkits.mplot3d import Axes3D
from numba import jit, prange
from ipywidgets import FloatSlider, IntSlider, Dropdown, VBox, HBox, Button, Output, Label
from IPython.display import display
import time

## NumPy Error Handling Configuration

In our script, we have configured how NumPy handles various types of floating-point errors using the `np.seterr` function. This configuration is crucial for controlling the script's behavior in the face of numerical issues, such as division by zero, overflows, underflows, and invalid operations. Here's what each setting means:

- `divide='warn'`: This setting issues a warning whenever a division by zero occurs. In floating-point arithmetic, dividing by zero yields an infinite result, which can be problematic in many calculations.

- `over='warn'`: This causes NumPy to generate a warning when an overflow occurs. An overflow happens when a variable is assigned a value beyond its maximum representable value, leading to loss of precision or incorrect results.

- `under='ignore'`: Underflows are ignored with this setting. An underflow occurs when a number is too small to be represented in the available precision, resulting in it being rounded down to zero.

- `invalid='warn'`: This setting triggers a warning for invalid operations. These are operations that yield a result that is not a well-defined number, like NaN (Not a Number), such as taking the square root of a negative number.

These settings help in identifying potential numerical issues during the execution of the script, ensuring that they do not go unnoticed, which could lead to incorrect conclusions or results.

In [38]:
# NumPy Error Handling Configuration
np.seterr(divide='warn', over='raise', under='ignore', invalid='warn')

# SymPy Global Evaluation Precision
# SymPy uses arbitrary precision, so you can set it as needed. For example, 50 decimal places:
sp.init_printing(use_unicode=True, precision=50)

# Defining Functions

In [39]:
# Define quantum gate-inspired effects
def pauli_x_gate(z, hbar):
    # Inspired by the Pauli X gate (bit-flip)
    return complex(z.imag, z.real) * hbar

def pauli_y_gate(z, hbar):
    # Inspired by the Pauli Y gate
    return complex(-z.imag, z.real) * hbar

def hadamard_gate(z, hbar):
    # Inspired by the Hadamard gate
    return (z.real + z.imag) + 1j * (z.real - z.imag) * hbar

def phase_shift_gate(z, hbar):
    # Inspired by the phase shift gate
    return np.exp(1j * hbar) * z

quantum_effects = {
    'Pauli X Gate': pauli_x_gate,
    'Pauli Y Gate': pauli_y_gate,
    'Hadamard Gate': hadamard_gate,
    'Phase Shift Gate': phase_shift_gate
}


In [43]:
def apply_quantum_effect(zx, zy, hbar, quantum_effect):
    z = zx + 1j * zy
    z = quantum_effect(z, hbar)
    return z.real, z.imag

@jit(parallel=True)
def quantum_julia_set(width, height, x_min, x_max, y_min, y_max, c, max_iter, hbar, quantum_effect):
    x = np.linspace(x_min, x_max, width)
    y = np.linspace(y_min, y_max, height)
    fractal = np.zeros((height, width), dtype=np.int64)

    for i in prange(height):
        for j in prange(width):
            zx, zy = x[j], y[i]
            iteration = 0
            while zx*zx + zy*zy < 4 and iteration < max_iter:
                zx, zy = apply_quantum_effect(zx, zy, hbar, quantum_effect)
                new_zx = zx*zx - zy*zy + c.real
                zy = 2*zx*zy + c.imag
                zx = new_zx
                iteration += 1
            fractal[i, j] = iteration

    return fractal

# Function to analyze and print quantum influence details
def analyze_quantum_influence(fractal, hbar):
    avg_iterations = np.mean(fractal)
    max_iterations = np.max(fractal)
    std_dev = np.std(fractal)
    high_iter_area = np.sum(fractal >= max_iterations * 0.8) / fractal.size
    low_iter_area = np.sum(fractal < max_iterations * 0.2) / fractal.size
    quantum_effect = np.sum(np.abs(np.gradient(fractal))) / fractal.size
    complexity_ratio = high_iter_area / low_iter_area

    print(f"Hbar (Quantum Factor): {hbar}")
    print(f"Average number of iterations: {avg_iterations:.2f}")
    print(f"Maximum number of iterations: {max_iterations}")
    print(f"Standard deviation of iterations: {std_dev:.2f}")
    print(f"Area with high iterations (>80% of max): {high_iter_area:.2f}")
    print(f"Area with low iterations (<20% of max): {low_iter_area:.2f}")
    print(f"Quantum Effect Measure: {quantum_effect:.2f}")
    print(f"Complexity Ratio (High/Low Iterations): {complexity_ratio:.2f}")

# Function to plot and optionally save the Julia set with sophisticated visuals
def plot_julia(fractal, load_time, save=False):
    fig = plt.figure(figsize=(18, 6))

    # Main Fractal Image
    ax1 = fig.add_subplot(131)
    ax1.imshow(fractal, cmap='viridis', interpolation='nearest', extent=[x_min.value, x_max.value, y_min.value, y_max.value])
    ax1.set_title("Quantum Julia Set")
    ax1.set_xlabel("Real Axis")
    ax1.set_ylabel("Imaginary Axis")

    # Gradient of the Fractal
    ax2 = fig.add_subplot(132)
    gradient = np.gradient(fractal.astype(float))
    gradient_magnitude = np.sqrt(gradient[0]**2 + gradient[1]**2)
    ax2.imshow(gradient_magnitude, cmap='magma', interpolation='nearest', extent=[x_min.value, x_max.value, y_min.value, y_max.value])
    ax2.set_title("Gradient Magnitude")
    ax2.set_xlabel("Real Axis")
    ax2.set_ylabel("Imaginary Axis")

    # 3D Surface Plot
    ax3 = fig.add_subplot(133, projection='3d')
    X, Y = np.meshgrid(np.linspace(x_min.value, x_max.value, fractal.shape[1]), np.linspace(y_min.value, y_max.value, fractal.shape[0]))
    ax3.plot_surface(X, Y, fractal, cmap='plasma')
    ax3.set_title("3D View of Julia Set")
    ax3.set_xlabel("Real Axis")
    ax3.set_ylabel("Imaginary Axis")
    ax3.set_zlabel("Iterations")

    plt.tight_layout()
    if save:
        plt.savefig('quantum_julia_set.png')
    plt.show()

# Function to handle button click for fractal generation with timing
output = Output()
loading_label = Label()

def on_generate_button_clicked(b):
    loading_label.value = "Generating fractal, please wait..."
    with output:
        output.clear_output()
        start_time = time.time()
        c = complex(c_real.value, c_imag.value)
        hbar_value = hbar.value
        selected_effect = quantum_effects[quantum_effect_dropdown.value]
        fractal = quantum_julia_set(800, 800, x_min.value, x_max.value, y_min.value, y_max.value, c, max_iter.value, hbar_value, selected_effect)
        end_time = time.time()
        plot_julia(fractal, end_time - start_time)
        analyze_quantum_influence(fractal, hbar_value)
    loading_label.value = ""

# Define interactive widgets
c_real = FloatSlider(min=-2.0, max=2.0, step=0.01, value=-0.8, description='Re(c):')
c_imag = FloatSlider(min=-2.0, max=2.0, step=0.01, value=0.156, description='Im(c):')
x_min = FloatSlider(min=-2.0, max=2.0, step=0.01, value=-1.5, description='x_min:')
x_max = FloatSlider(min=-2.0, max=2.0, step=0.01, value=1.5, description='x_max:')
y_min = FloatSlider(min=-2.0, max=2.0, step=0.01, value=-1.5, description='y_min:')
y_max = FloatSlider(min=-2.0, max=2.0, step=0.01, value=1.5, description='y_max:')
max_iter = IntSlider(min=1, max=1000, step=1, value=300, description='Max Iter:')
hbar = FloatSlider(min=0.0, max=2.0, step=0.01, value=1.0, description='Quantum Factor:')

# Dropdown menu for selecting the quantum effect
quantum_effect_dropdown = Dropdown(
    options=list(quantum_effects.keys()),
    value='Pauli X Gate',
    description='Quantum Effect:'
)

# Create a button to generate the fractal
generate_button = Button(description="Generate Fractal")
generate_button.on_click(on_generate_button_clicked)

# Arrange the UI elements
ui = VBox([
    HBox([c_real, c_imag]),
    HBox([x_min, x_max]),
    HBox([y_min, y_max]),
    HBox([max_iter, hbar]),
    quantum_effect_dropdown,
    generate_button,
    loading_label,
    output
])

# Display the UI
display(ui)

<ipython-input-43-a8444fbbf9d2>:6: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(parallel=True)
